In [8]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, StopLossRequest, TakeProfitRequest
from alpaca.trading.enums import OrderSide, TimeInForce, AssetClass
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame

import pandas as pd
from apscheduler.schedulers.blocking import BlockingScheduler
from datetime import datetime, timedelta

In [9]:
### Code given from the Alpaca website info

TRADE_API_KEY="PKHZ8KJAZ8NRZ18EUKZZ"
TRADE_API_SECRET="NA07lslGjfRhDbbjzPdqjBnUv6nlrsy1YQIUCVet"

#### We use paper environment for this example ####
PAPER=True
TRADE_API_URL="https://paper-api.alpaca.markets"
TRADE_API_WSS=None
DATA_API_URL=None
OPTION_STREAM_DATA_WSS=None

api_key = TRADE_API_KEY
secret_key = TRADE_API_SECRET
paper = PAPER
trade_api_url = TRADE_API_URL

trade_client = TradingClient(TRADE_API_KEY, TRADE_API_SECRET, paper=PAPER, url_override= TRADE_API_URL)
data_client = StockHistoricalDataClient(TRADE_API_KEY, TRADE_API_SECRET)


In [38]:
# Fetch Historical Data 

def get_stock_data(symbol, timeframe="15Min", limit=50):
    # Create a StockBarsRequest with the necessary parameters
    request_params = StockBarsRequest(
        symbol_or_symbols=symbol,
        timeframe=TimeFrame.Minute,
        limit=limit
    )
    
    # Fetch stock bars from Alpaca
    bars, metadata = data_client.get_stock_bars(request_params)
    
    # Convert the bars into a pandas DataFrame
    df = pd.DataFrame({
        'open': [bar.o for bar in bars],
        'high': [bar.h for bar in bars],
        'low': [bar.l for bar in bars],
        'close': [bar.c for bar in bars]
    })
    
    return df

In [39]:
# Signal generator logic

def signal_generator(df):
    open = df["open"].iloc[-1]
    close = df["close"].iloc[-1]
    previous_open = df["open"].iloc[-2]
    previous_close = df["close"].iloc[-2]
    
    # Bearish Pattern
    if (open > close and 
        previous_open < previous_close and 
        close < previous_open and
        open >= previous_close):
        return "sell"
    
     # Bullish Pattern
    elif (open < close and 
          previous_open > previous_close and 
          close > previous_open and
          open <= previous_close):
        return "buy"
    
    # No clear pattern
    else:
        return "hold"

In [40]:
def trading_job():
    symbol = "AAPL"  # Stock to trade
    bars = get_stock_data(symbol)
    signal = signal_generator(bars)

    # Define trade parameters
    last_price = bars["close"].iloc[-1]
    risk_tolerance = 2.0  # Stop-loss/take-profit multiplier
    trade_size = 1  # Number of shares

    range_previous_candle = abs(bars["high"].iloc[-2] - bars["low"].iloc[-2])
    SLBuy = last_price - range_previous_candle
    TPBuy = last_price + (range_previous_candle * risk_tolerance)
    SLSell = last_price + range_previous_candle
    TPSell = last_price - (range_previous_candle * risk_tolerance)

    print(f"Signal: {signal}, Last Price: {last_price}")
    print(f"Buy TP: {TPBuy}, Buy SL: {SLBuy}, Sell TP: {TPSell}, Sell SL: {SLSell}")


        # Execute trades based on the signal
    try:
        if signal == "buy":
            # Place buy market order
            market_order = MarketOrderRequest(
                symbol=symbol,
                qty=trade_size,
                side=OrderSide.BUY,
                time_in_force=TimeInForce.GTC
            )
            trade_client.submit_order(market_order)
            print(f"Buy order placed for {symbol}")
        elif signal == "sell":
            # Place sell market order
            market_order = MarketOrderRequest(
                symbol=symbol,
                qty=trade_size,
                side=OrderSide.SELL,
                time_in_force=TimeInForce.GTC
            )
            trade_client.submit_order(market_order)
            print(f"Sell order placed for {symbol}")
        else:
            print("No trade executed.")
    except Exception as e:
        print(f"Error executing trade: {e}")


In [41]:
# Scheduler to run the bot

#scheduler = BlockingScheduler(timezone="America/Los_Angeles")
#scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='06-15', minute='0,15,30,45')  # Adjust as needed
#scheduler.start()

trading_job()

ValueError: not enough values to unpack (expected 2, got 1)